# Librerías a instalar que no vienen por default
Correrlos en cmd para instalarlos<br>  pip install BeautifulSoup4<br>  pip install requests<br>  pip install selenium<br>

Hay que descargar el web driver de tu navegador

# Importar librerías

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import pandas as pd

# Código

## Funciones

In [5]:
def check_exists_by_xpath(webdriver, xpath):
    """
        Regresa True si existe el elemento con el xpath dado, sino False
    """
    try:
        webdriver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True


def selectOpc(driver,xpathDiv1, xpathDivOpc, pos):
    """
        Selecciona la opción localizada en la posición pos de xpathDivOpc
        driver: driver del navegador
        xpathDiv1:
    """
    #Damos click para que aparezca el div de opciones
        # Corrijo bug -- FDM
    element = driver.find_element_by_xpath(xpathDiv1)
    driver.execute_script("arguments[0].click();", element)
    #Div de opciones
    opcs = driver.find_element_by_xpath(xpathDivOpc).find_elements_by_tag_name("div") 
    #Damos clic a la opción
    try:
        driver.execute_script("arguments[0].click();",  opcs[pos])
    except Exception as e:
        print(str(e))
        print(f"Element {opcs[pos]} not clickable")


def getSizeByTag(driver, xpath, tag):
    """
        Dada un xpath busca todos los elementos con el tag dado y regresa el tamaño
    """
    return len(driver.find_element_by_xpath(xpath).find_elements_by_tag_name(tag))


def doClick(driver, xpath):
    try:
        driver.find_element_by_xpath(xpath).click()
    except:
        print(f"{xpath} not clickable")


def getElementsByTag(driver, xpath, tag):
    element = driver.find_element_by_xpath(xpath).find_elements_by_tag_name(tag)
    return element

def checkTable(driver, oneTable):
    rowsPath = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody'
    btnCerrarVisor = '//*[@id="modalPDF"]/div/div/div[3]/button'    
    visorPDF = '//*[@id="ContentPlaceHolder1_framePDF"]'
    rows = getElementsByTag(driver, rowsPath, "tr")
    pdf_links = []                    
    if oneTable:
        nr = len(rows) 
    else:
        nr = len(rows)-1

    # Get table data with beautiful soup
    html = driver.page_source
    sopa = BeautifulSoup(html, 'html.parser')
    tds = sopa.find_all("td", {"style": ["width:7%;", "width:15%;", "width:9%;", "width:12%;", "width:2%;"]})
    titles = sopa.find_all("th")
    lista_ds = []
    i = 0
    while i < len(tds):
        d = {}
        d[titles[1].text] = tds[i].text
        d[titles[2].text] = tds[i + 1].text
        d[titles[3].text] = tds[i + 2].text
        d[titles[4].text] = tds[i + 3].text
        d[titles[5].text] = tds[i + 4].text
        d[titles[6].text] = tds[i + 5].text
        d[titles[7].text] = tds[i + 6].text
        lista_ds.append(d)
        i += 7
    df = pd.DataFrame.from_dict(lista_ds)
    j = 0
    while (j < len(df['Sentencia'])):
        rows = getElementsByTag(driver, rowsPath, "tr")

        cols = rows[j].find_elements_by_tag_name("td")
        nCol = len(cols)
        
        # for y in range(nCol-1):
            # print(cols[y].get_attribute("innerHTML"))
                                
        #Abrir pdf para obtener link
        try:
            cols[nCol-1].click()
        except Exception as e:
            print(str(e))
            print(f"Element {cols[nCol-1]} not clickable")                    
        #Esperamos a que cargue
        time.sleep(4)        
                           
        pdf_link = driver.find_element_by_xpath(visorPDF).get_attribute('src')
        pdf_links.append(pdf_link)

        #Cerrar el visor
        time.sleep(4)
        doClick(driver, btnCerrarVisor)
        j += 1                        

        #Boton descargar
        #//*[@id="download"]    
    time.sleep(8)
    df[titles[7].text] = pdf_links
    print(df)
    return df


## Web scraping

In [6]:
#Variar Tipo de Juzgado, Delitos-Modalidades,  Criterios , Tipo de resolución
#Checar el comentario del primer for (i1)

driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.pjenl.gob.mx/SentenciasPublicas/Modulos/Penales.aspx")
divJuzgado = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[1]/div/div/div[1]'
divOpcJuzgado = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[1]/div/div/div[2]/div'
divDelitos = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[2]/div/div/div[1]'
divOpcDelitos = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[2]/div/div/div[2]/div'
divCriterios = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[5]/div/div/div[1]'
divOpcCriterios = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[5]/div/div/div[2]/div'
divResolucion = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[6]/div/div/div[1]'
divOpcResolucion = '//*[@id="ContentPlaceHolder1_ctl00"]/div[2]/div/div[1]/div[6]/div/div/div[2]/div'
btnBuscar = '//*[@id="ContentPlaceHolder1_BtnBuscar"]'
tabla = '//*[@id="ContentPlaceHolder1_gdvPenal"]/thead/tr/th[2]'
indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr'

lista_dfs = []

# Tipo de juzgado ----------------------------------------------------
selectOpc(driver,divJuzgado,divOpcJuzgado, 0)
t1 = getSizeByTag(driver, divOpcJuzgado, "div")

#Si ya es la versión final hay que cambiar el for para que empiece con 1.

#Empieza en 2 para probar
for i1 in range(2,t1):                                     
    selectOpc(driver,divJuzgado,divOpcJuzgado, i1)   

    #Delitos - Modalidades  ----------------------------------------------  
    selectOpc(driver, divDelitos, divOpcDelitos, 0)        
    t2 = getSizeByTag(driver, divOpcDelitos,"div")
    time.sleep(1)

    #Las primeras 7 son Ver todos y opciones vacías
    for i2 in range(7,t2):                           
        selectOpc(driver, divDelitos, divOpcDelitos, i2)
        time.sleep(1)        

        #Criterios Específicos ----------------------------------------------      
        selectOpc(driver, divCriterios ,divOpcCriterios , 0)                              
        t3 = getSizeByTag(driver, divOpcCriterios,"div")

        for i3 in range(t3):             
            selectOpc(driver, divCriterios ,divOpcCriterios , i3)               

            #Tipo resolución --------------------------------------------------                    
            selectOpc(driver, divResolucion, divOpcResolucion, 0)
            t4 = getSizeByTag(driver, divOpcResolucion,"div")

            for i4 in range(1, t4):                      
                selectOpc(driver, divResolucion, divOpcResolucion, i4)

                #Hacemos consulta -----------------------------------------------
                doClick(driver, btnBuscar)
                time.sleep(3)
                #Pendiente Checar si ya dio el resultado en lugar de poner 3 segundos    
                                
                if check_exists_by_xpath(driver, tabla):
                    #Hay tabla

                    if check_exists_by_xpath(driver, '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/                                                          tbody/tr'):        
                        first = True
                        while True:
                            ind = getElementsByTag(driver,indexList, "a")
                            if ind[-1].get_attribute("text") != "...":
                                break
                            # First sirve para saber en que index empezar, porque solo las primeras 10 son de las forma                               [1,..., 10, "..."]
                            # En ese caso empezamos en 0.
                            # Las demás serán  ["...", n, n+1, ..., n+9, "..."] entonces hay que empezar en 1, sino se                                regresa a las anteriores
                            if first:
                                i = 0
                                first=False
                            else:
                                i = 1
                            start = True
                            while ind[-1].get_attribute("text") == "...":  
                                df1 = checkTable(driver,False)
                                lista_dfs.append(df1)
                                # Por alguna razón después de recorrer la tabla cambia el html y el encabezado se hace                                    parte de las filas y deja de existir el thead
                                indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[22]/td/table/tbody/tr'                                
                                try: 
                                    ind = getElementsByTag(driver,indexList, "a")
                                    ind[i].click()
                                except:
                                    pass           
                                if(ind[i].get_attribute("text") == "..."):
                                    break                                
                                i += 1
                                #Este es opcional, hay que buscar una manera de saber cuando ya cargó la página                
                                time.sleep(30)
                                driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)  
                                #Después de que cambiamos de página el html regresa a lo normal y el thead ya existe de nuevo
                                indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr'
                                ind = getElementsByTag(driver,indexList, "a")              
                            time.sleep(30)
                        #Ya estamos en la última página de sentencias
                        df2 = checkTable(driver,False)
                        lista_dfs.append(df2)                                                        
                    else:
                        df3 = checkTable(driver,True)
                        lista_dfs.append(df3)
                    print(lista_dfs)               


Message: element click intercepted: Element <td style="width:2%;">...</td> is not clickable at point (1340, 615). Other element would receive the click: <div class="modal-backdrop fade in">...</div>
  (Session info: chrome=89.0.4389.90)

Element <selenium.webdriver.remote.webelement.WebElement (session="a805fd13acf9be020bb8f0f8507c547e", element="606e2d73-d3bc-4d08-89da-fb6f35cbec56")> not clickable
//*[@id="modalPDF"]/div/div/div[3]/button not clickable
Message: element click intercepted: Element <td style="width:2%;">...</td> is not clickable at point (1340, 615). Other element would receive the click: <div class="modal-backdrop fade in">...</div>
  (Session info: chrome=89.0.4389.90)

Element <selenium.webdriver.remote.webelement.WebElement (session="a805fd13acf9be020bb8f0f8507c547e", element="c38db5bf-8082-4a9f-b1ab-872d827461aa")> not clickable
//*[@id="modalPDF"]/div/div/div[3]/button not clickable
//*[@id="modalPDF"]/div/div/div[3]/button not clickable
//*[@id="modalPDF"]/div/di

KeyboardInterrupt: 

In [7]:
lista_dfs

[      Fecha Emisión Fecha Publicación                   Delitos-Modalidades  \
 0     18/marzo/2021     18/marzo/2021                    ROBO CON VIOLENCIA   
 1     10/marzo/2021     12/marzo/2021                    ATENTADOS AL PUDOR   
 2     04/marzo/2021     19/marzo/2021           LESIONES DOLOSAS CALIFICADA   
 3     03/marzo/2021     10/marzo/2021                                         
 4     03/marzo/2021     10/marzo/2021                                         
 5   28/febrero/2021     02/marzo/2021                                         
 6   26/febrero/2021     09/marzo/2021                                         
 7   26/febrero/2021     02/marzo/2021                                         
 8   26/febrero/2021     02/marzo/2021                                         
 9   26/febrero/2021     02/marzo/2021                                         
 10  26/febrero/2021     01/marzo/2021                     HOMICIDIO CULPOSO   
 11  26/febrero/2021     02/marzo/2021  

In [8]:
df_final = pd.concat(lista_dfs, ignore_index = True)
df_final

,Fecha Emisión,Fecha Publicación,Delitos-Modalidades,Juez,Sentido del Fallo,Descripción o Criterios Judiciales,Sentencia
0,18/marzo/2021,18/marzo/2021,ROBO CON VIOLENCIA,MARÍA DEL SOCORRO PÉREZ CÓRDOVA(Elabora),CONDENATORIO,SENTENCIA CONDENATORIA,about:blank
1,10/marzo/2021,12/marzo/2021,ATENTADOS AL PUDOR,PRIMITIVO JESÚS GUTIÉRREZ VÉLEZ(Elabora),CONDENATORIO,SENTENCIA CONDENATORIA,about:blank
2,04/marzo/2021,19/marzo/2021,LESIONES DOLOSAS CALIFICADA,GRACIELA CÁZARES CHARLES(Elabora),CONDENATORIO,SENTENCIA CONDENATORIA,https://www.pjenl.gob.mx/SentenciasPublicas/PD...
3,03/marzo/2021,10/marzo/2021,,,SE CONFIRMA,,https://www.pjenl.gob.mx/SentenciasPublicas/PD...
4,03/marzo/2021,10/marzo/2021,,"JULIANA CRUZ GUILLÉN(Vocero), AÍDA ARACELI REY...",SE REVOCA,,https://www.pjenl.gob.mx/SentenciasPublicas/PD...
5,28/febrero/2021,02/marzo/2021,,,SE CONFIRMA,,https://www.pjenl.gob.mx/SentenciasPublicas/PD...
6,26/febrero/2021,09/marzo/2021,,JULIANA CRUZ GUILLÉN(Elabora),SE CONFIRMA,,https://www.pjenl.gob.mx/SentenciasPublicas/PD...
7,26/febrero/2021,02/marzo/2021,,JULIANA CRUZ GUILLÉN(Elabora),SE REVOCA,,https://www.pjenl.gob.mx/SentenciasPublicas/PD...
8,26/febrero/2021,02/marzo/2021,,,SE CONFIRMA,,https://www.pjenl.gob.mx/SentenciasPublicas/PD...
9,26/febrero/2021,02/marzo/2021,,,SE CONFIRMA,,https://www.pjenl.gob.mx/SentenciasPublicas/PD...


In [20]:
driver.quit()

## Prueba para cuando hay múltiples páginas de sentencias

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.pjenl.gob.mx/SentenciasPublicas/Modulos/Penales.aspx")
indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr'            

print("Entra")
if check_exists_by_xpath(driver,'//*[@id="ContentPlaceHolder1_gdvPenal"]/thead/tr/th[2]'):
    #Hay tabla    
    if check_exists_by_xpath(driver, '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr'):        
        first = True
        while True:
            ind = getElementsByTag(driver,indexList, "a")
            if ind[-1].get_attribute("text") != "...":
                break
            #First sirve para saber en que index empezar, porque solo las primeras 10 son de las forma [1,..., 10, "..."]
            #En ese caso empezamos en 0.
            #Las demás serán  ["...", n, n+1, ..., n+9, "..."] entonces hay que empezar en 1, sino se regresa a las anteriores
            if first:
                i = 0
                first=False
            else:
                i=1
            start = True
            while ind[-1].get_attribute("text") == "...":  
                checkTable(driver,False) 
                #Por alguna razón después de recorrer la tabla cambia el html y el encabezado se hace parte
                # de las filas y deja de existir el thead
                indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[22]/td/table/tbody/tr'                                
                ind = getElementsByTag(driver,indexList, "a")
                ind[i].click()           
                if(ind[i].get_attribute("text") == "..."):
                    break                                
                i+=1
                #Este es opcional, hay que buscar una manera de saber cuando ya cargó la página                
                time.sleep(30)
                driver.find_element_by_tag_name('body').send_keys(Keys.CONTROL + Keys.HOME)  
                #Después de que cambiamos de página el html regresa a lo normal y el thead ya existe de nuevo
                indexList = '//*[@id="ContentPlaceHolder1_gdvPenal"]/tbody/tr[21]/td/table/tbody/tr'
                ind = getElementsByTag(driver,indexList, "a")              
            time.sleep(30)
        #Ya estamos en la última página de sentencias
        checkTable(driver,False)
        
                
    else:
        checkTable(driver,True)
